In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled pr

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

# =========================================================
# 1. 환경 설정 및 상수 정의
# =========================================================

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 키포인트 리스트
KEYPOINTS = [
    "Nose", "Left Eye", "Right Eye",
    "Left Shoulder", "Right Shoulder",
    "Left Hip", "Right Hip",
    "Left Knee", "Right Knee",
    "Left Ankle", "Right Ankle",
    "Left Foot", "Right Foot"
]

# MediaPipe 랜드마크 상수 매핑
MP_MAP = {
    "Nose": mp_pose.PoseLandmark.NOSE,
    "Left Eye": mp_pose.PoseLandmark.LEFT_EYE,
    "Right Eye": mp_pose.PoseLandmark.RIGHT_EYE,
    "Left Shoulder": mp_pose.PoseLandmark.LEFT_SHOULDER,
    "Right Shoulder": mp_pose.PoseLandmark.RIGHT_SHOULDER,
    "Left Hip": mp_pose.PoseLandmark.LEFT_HIP,
    "Right Hip": mp_pose.PoseLandmark.RIGHT_HIP,
    "Left Knee": mp_pose.PoseLandmark.LEFT_KNEE,
    "Right Knee": mp_pose.PoseLandmark.RIGHT_KNEE,
    "Left Ankle": mp_pose.PoseLandmark.LEFT_ANKLE,
    "Right Ankle": mp_pose.PoseLandmark.RIGHT_ANKLE,
    "Left Foot": mp_pose.PoseLandmark.LEFT_FOOT_INDEX,
    "Right Foot": mp_pose.PoseLandmark.RIGHT_FOOT_INDEX
}

# ✅ [수정] 각도계산_real_real.ipynb와 동일하게 키 추가
SIDE_VIEW_COLUMNS = [
    "knee_left", "knee_right", "knee_mean",
    "hip_flexion_left", "hip_flexion_right", "hip_flexion_mean",
    "ankle_dorsi_left", "ankle_dorsi_right", "ankle_dorsi_mean",
    "trunk_inclination",
    "tibia_inclination_left", "tibia_inclination_right", "tibia_inclination_mean",
    "face_shoulder_hip_left",  # <--- NEW
    "face_shoulder_hip_right", # <--- NEW
]

FRONT_VIEW_COLUMNS = [
    "fppa_left", "fppa_right", "fppa_mean",
    "pelvic_tilt",
    "trunk_lateral_tilt",
    "knee_width_ratio",
    "head_pitch_proxy",
]

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [2]:
# =========================================================
# 2. 기하학 헬퍼 함수 (각도계산_real_real.ipynb 원본 동기화)
# =========================================================

def get_pt(pts, name):
    p = pts.get(name)
    if p is None:
        return None
    return np.array([p["x"], p["y"]], dtype=float)

def l2(v):
    return float(np.linalg.norm(v))

def angle_between(v1, v2, eps=1e-8):
    n1, n2 = l2(v1), l2(v2)
    if n1 < eps or n2 < eps:
        return None
    cos = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cos)))

def joint_angle(A, B, C):
    """ 각도 ∠ABC (B가 꼭짓점) """
    if A is None or B is None or C is None:
        return None
    v1 = A - B
    v2 = C - B
    return angle_between(v1, v2)

def midpoint(P, Q):
    if P is None or Q is None:
        return None
    return (P + Q) / 2

def dist(P, Q):
    if P is None or Q is None:
        return None
    return float(np.linalg.norm(P - Q))

V_UP = np.array([0.0, -1.0], dtype=float)   # 화면 위쪽
H_RIGHT = np.array([1.0, 0.0], dtype=float) # 화면 오른쪽


# =========================
# 3. "모든 각도" 계산 함수 (원본 로직 100% 반영)
# =========================
def compute_all_angles(pts, existing_angles=None):
    """
    pts: { "Nose": {"x":..., "y":...}, ... }
    existing_angles: 이미 들어있는 angles (knee_* 등)
    return: angles dict (left/right/mean 다 계산)
    """
    ang = dict(existing_angles) if existing_angles else {}

    # ---- points ----
    Nose = get_pt(pts, "Nose")
    LEye = get_pt(pts, "Left Eye")
    REye = get_pt(pts, "Right Eye")

    LS = get_pt(pts, "Left Shoulder")
    RS = get_pt(pts, "Right Shoulder")
    LH = get_pt(pts, "Left Hip")
    RH = get_pt(pts, "Right Hip")
    LK = get_pt(pts, "Left Knee")
    RK = get_pt(pts, "Right Knee")
    LA = get_pt(pts, "Left Ankle")
    RA = get_pt(pts, "Right Ankle")
    LF = get_pt(pts, "Left Foot")
    RF = get_pt(pts, "Right Foot")

    shoulder_mid = midpoint(LS, RS)
    hip_mid = midpoint(LH, RH)

    # ======================================================
    # (1) Knee Flexion
    # ======================================================
    knee_left = ang.get("knee_left")
    knee_right = ang.get("knee_right")
    knee_mean = ang.get("knee_mean")

    if knee_left is None and LH is not None and LK is not None and LA is not None:
        knee_left = joint_angle(LH, LK, LA)
        ang["knee_left"] = knee_left

    if knee_right is None and RH is not None and RK is not None and RA is not None:
        knee_right = joint_angle(RH, RK, RA)
        ang["knee_right"] = knee_right

    if knee_mean is None and knee_left is not None and knee_right is not None:
        ang["knee_mean"] = float((knee_left + knee_right) / 2)

    # ======================================================
    # (2) Hip Flexion
    # ======================================================
    trunk = None if (shoulder_mid is None or hip_mid is None) else (shoulder_mid - hip_mid)

    hip_left = None
    hip_right = None

    if trunk is not None and LK is not None and hip_mid is not None:
        thigh_L = LK - hip_mid
        hip_left = angle_between(trunk, thigh_L)
    if trunk is not None and RK is not None and hip_mid is not None:
        thigh_R = RK - hip_mid
        hip_right = angle_between(trunk, thigh_R)

    ang["hip_flexion_left"] = hip_left
    ang["hip_flexion_right"] = hip_right
    ang["hip_flexion_mean"] = None if (hip_left is None or hip_right is None) \
                                   else float((hip_left + hip_right) / 2)

    # ======================================================
    # (3) Ankle Dorsiflexion
    # ======================================================
    ankle_left = joint_angle(LK, LA, LF) if (LK is not None and LA is not None and LF is not None) else None
    ankle_right = joint_angle(RK, RA, RF) if (RK is not None and RA is not None and RF is not None) else None

    ang["ankle_dorsi_left"] = ankle_left
    ang["ankle_dorsi_right"] = ankle_right
    ang["ankle_dorsi_mean"] = None if (ankle_left is None or ankle_right is None) \
                                   else float((ankle_left + ankle_right) / 2)

    # ======================================================
    # (4) Trunk Inclination
    # ======================================================
    ang["trunk_inclination"] = angle_between(trunk, V_UP) if trunk is not None else None

    # ======================================================
    # (5) Tibia Inclination
    # ======================================================
    tibia_left = angle_between((LK-LA), V_UP) if (LK is not None and LA is not None) else None
    tibia_right = angle_between((RK-RA), V_UP) if (RK is not None and RA is not None) else None

    ang["tibia_inclination_left"] = tibia_left
    ang["tibia_inclination_right"] = tibia_right
    ang["tibia_inclination_mean"] = None if (tibia_left is None or tibia_right is None) \
                                         else float((tibia_left + tibia_right) / 2)

    # ======================================================
    # (6) FPPA
    # ======================================================
    fppa_left = angle_between((LH-LK), (LA-LK)) if (LH is not None and LK is not None and LA is not None) else None
    fppa_right = angle_between((RH-RK), (RA-RK)) if (RH is not None and RK is not None and RA is not None) else None

    ang["fppa_left"] = fppa_left
    ang["fppa_right"] = fppa_right
    ang["fppa_mean"] = None if (fppa_left is None or fppa_right is None) \
                            else float((fppa_left + fppa_right) / 2)

    # ======================================================
    # (7) Pelvic Tilt
    # ======================================================
    pelvis_vec = (RH - LH) if (LH is not None and RH is not None) else None
    ang["pelvic_tilt"] = angle_between(pelvis_vec, H_RIGHT) if pelvis_vec is not None else None

    # ======================================================
    # (8) Trunk Lateral Tilt
    # ======================================================
    shoulder_line = (RS - LS) if (LS is not None and RS is not None) else None
    ang["trunk_lateral_tilt"] = angle_between(shoulder_line, H_RIGHT) if shoulder_line is not None else None

    # ======================================================
    # (9) Knee Width Ratio
    # ======================================================
    d_knee = dist(LK, RK)
    d_ankle = dist(LA, RA)

    if d_knee is None or d_ankle is None or d_ankle == 0:
        ang["knee_width_ratio"] = None
    else:
        ang["knee_width_ratio"] = float(d_knee / d_ankle)

    # ======================================================
    # (10) Head Pitch Proxy (위/아래 정면응시)
    # ======================================================
    head_pitch_proxy = None
    if Nose is not None and LEye is not None and REye is not None and LS is not None and RS is not None:
        face_mid = (Nose + LEye + REye) / 3.0
        shoulder_mid2 = (LS + RS) / 2.0
        d_face_shoulder = np.linalg.norm(face_mid - shoulder_mid2)

        shoulder_width = np.linalg.norm(RS - LS)
        if shoulder_width > 1e-6:
            head_pitch_proxy = float(d_face_shoulder / shoulder_width)

    ang["head_pitch_proxy"] = head_pitch_proxy

    # ======================================================
    # (11) Face_mid - Shoulder - Hip angle (Left/Right)
    #     face_mid = (Nose + LeftEye + RightEye) / 3
    #     left  : ∠(face_mid - LeftShoulder - LeftHip)
    #     right : ∠(face_mid - RightShoulder - RightHip)
    # ======================================================
    face_shoulder_hip_left = None
    face_shoulder_hip_right = None

    if Nose is not None and LEye is not None and REye is not None:
        face_mid = (Nose + LEye + REye) / 3.0

        # ✅ 왼쪽 각도: 꼭짓점 = Left Shoulder
        if face_mid is not None and LS is not None and LH is not None:
            face_shoulder_hip_left = joint_angle(face_mid, LS, LH)

        # ✅ 오른쪽 각도: 꼭짓점 = Right Shoulder
        if face_mid is not None and RS is not None and RH is not None:
            face_shoulder_hip_right = joint_angle(face_mid, RS, RH)

    ang["face_shoulder_hip_left"] = face_shoulder_hip_left
    ang["face_shoulder_hip_right"] = face_shoulder_hip_right

    return ang

In [3]:
# =========================================================
# 4. 클래스 정의
# =========================================================

class VideoPoseExtractor:
    """
    [책임] 영상 파일에서 MediaPipe를 이용해 좌표(Coords)와 신뢰도(Visibility)를 추출합니다.
    """
    def __init__(self, video_path):
        self.video_path = video_path
        self.width = 0
        self.height = 0
        self.fps = 0

    def extract_landmarks(self):
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print(f"❌ Error: Cannot open video {self.video_path}")
            return None, None, None

        self.width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = cap.get(cv2.CAP_PROP_FPS)

        coords_list, vis_list, raw_list = [], [], []

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=1) as pose:
            while True:
                ret, frame = cap.read()
                if not ret: break

                results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                f_coords = np.full((len(KEYPOINTS), 2), np.nan, dtype=np.float32)
                f_vis = np.full((len(KEYPOINTS),), 0.0, dtype=np.float32)

                if results.pose_landmarks:
                    raw_list.append(results.pose_landmarks)
                    for i, key in enumerate(KEYPOINTS):
                        lm = results.pose_landmarks.landmark[MP_MAP[key]]
                        f_coords[i] = [lm.x * self.width, lm.y * self.height]
                        f_vis[i] = lm.visibility
                else:
                    raw_list.append(None)

                coords_list.append(f_coords)
                vis_list.append(f_vis)

        cap.release()
        return np.array(coords_list), np.array(vis_list), raw_list


class VideoVisualizer:
    """
    [책임] 원본 영상 위에 스켈레톤을 그려 저장합니다.
    """
    def save_skeleton_video(self, input_path, output_path, raw_landmarks):
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened(): return

        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

        idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            if idx < len(raw_landmarks) and raw_landmarks[idx]:
                mp_drawing.draw_landmarks(
                    frame, raw_landmarks[idx], mp_pose.POSE_CONNECTIONS,
                    mp_drawing_styles.get_default_pose_landmarks_style()
                )
            out.write(frame)
            idx += 1

        cap.release()
        out.release()
        print(f"🎥 [Saved] 스켈레톤 영상 저장 완료: {output_path}")


class PoseAnalyzer:
    """
    [책임] 좌표 데이터를 기반으로 각도를 계산하고, 가려짐 처리 및 데이터 필터링을 수행합니다.
    """
    def __init__(self, smooth_window=5):
        self.smooth_window = smooth_window
        self.replacement_logs = []
        self.kp_idx = {k: i for i, k in enumerate(KEYPOINTS)}

    def smooth_signal(self, data):
        if len(data) < self.smooth_window: return data
        window = np.ones(self.smooth_window) / self.smooth_window
        return np.convolve(data, window, mode='same')

    def _array_to_pts_dict(self, frame_coords):
        pts = {}
        for i, k in enumerate(KEYPOINTS):
            if not np.isnan(frame_coords[i]).any():
                pts[k] = {"x": frame_coords[i][0], "y": frame_coords[i][1]}
        return pts

    def calculate_angles_and_filter(self, coords, vis, view_mode='side', vis_threshold=0.6):
        """
        1. 20개 전체 각도 계산 (compute_all_angles 사용)
        2. 가려짐 처리 (Side View일 때 Knee Mean 보정)
        3. view_mode에 맞는 각도만 필터링하여 반환
        """
        records = []
        self.replacement_logs = []

        idx_lk = self.kp_idx['Left Knee']
        idx_rk = self.kp_idx['Right Knee']

        for t in range(len(coords)):
            # 1. 계산
            pts = self._array_to_pts_dict(coords[t])
            angles = compute_all_angles(pts)

            # 2. 가려짐 처리
            if view_mode == 'side':
                vis_l = vis[t][idx_lk]
                vis_r = vis[t][idx_rk]
                knee_l = angles.get('knee_left')
                knee_r = angles.get('knee_right')
                smart_mean = angles.get('knee_mean')

                # 왼쪽 양호, 오른쪽 불량 -> 왼쪽 사용
                if (knee_l is not None and vis_l >= vis_threshold) and (knee_r is None or vis_r < vis_threshold):
                    smart_mean = knee_l
                    self.replacement_logs.append(f"Frame {t}: Right Knee Hidden (Vis {vis_r:.2f}) -> Replaced with Left")
                # 오른쪽 양호, 왼쪽 불량 -> 오른쪽 사용
                elif (knee_r is not None and vis_r >= vis_threshold) and (knee_l is None or vis_l < vis_threshold):
                    smart_mean = knee_r
                    self.replacement_logs.append(f"Frame {t}: Left Knee Hidden (Vis {vis_l:.2f}) -> Replaced with Right")

                angles['knee_mean'] = smart_mean

            # Rep Detector용 기본값
            if angles.get('knee_mean') is None:
                angles['knee_mean'] = 180.0

            records.append(angles)

        df_all = pd.DataFrame(records)

        # 3. 필터링
        if view_mode == 'side':
            target_cols = SIDE_VIEW_COLUMNS
        elif view_mode == 'front':
            target_cols = FRONT_VIEW_COLUMNS
        else:
            raise ValueError("view_mode must be 'side' or 'front'")

        return df_all, df_all[target_cols]

    def resample_rep(self, df_rep, target_len=64):
        """Rep 데이터를 64프레임으로 정규화(Resampling)"""
        if df_rep.empty: return df_rep
        x_old = np.linspace(0, 1, len(df_rep))
        x_new = np.linspace(0, 1, target_len)
        df_new = pd.DataFrame()

        for col in df_rep.columns:
            vals = df_rep[col].values.astype(float)
            mask = np.isnan(vals)
            vals[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), vals[~mask])
            df_new[col] = np.interp(x_new, x_old, vals)
        return df_new


class RepDetector:
    """
    [책임] 각도 시퀀스에서 스쿼트 동작(Rep)을 감지합니다.
    """
    def __init__(self, min_delta=10.0, min_frames=4):
        self.min_delta = min_delta
        self.min_frames = min_frames

    def detect(self, seq):
        if np.isnan(seq).all(): return []
        peaks, bottoms = [], []

        # 극점 찾기
        for t in range(1, len(seq)-1):
            if seq[t] > seq[t-1] and seq[t] >= seq[t+1]: peaks.append(t)
            elif seq[t] < seq[t-1] and seq[t] <= seq[t+1]: bottoms.append(t)

        reps = []
        peaks = np.array(peaks)

        # Rep 구성
        for b in bottoms:
            prev = peaks[peaks < b]
            nxt = peaks[peaks > b]
            if len(prev) > 0 and len(nxt) > 0:
                s, e = prev[-1], nxt[0]
                if (seq[s] - seq[b]) > self.min_delta and (e - s) > self.min_frames:
                    if not reps or reps[-1]['end'] != e:
                        reps.append({'start': s, 'bottom': b, 'end': e, 'min_angle': seq[b]})
        return reps

In [4]:
# =========================================================
# 5. 메인 프로세스
# =========================================================

def process_user_video(video_path, view_mode='side'):
    print(f"\n🎬 [Start] 영상 분석 시작 (Mode: {view_mode}): {os.path.basename(video_path)}")

    # 1. 추출
    extractor = VideoPoseExtractor(video_path)
    coords, vis, raw_lm = extractor.extract_landmarks()
    if coords is None: return []

    # 2. 시각화
    vis_saver = VideoVisualizer()
    output_vid = os.path.splitext(video_path)[0] + f"_{view_mode}_skeleton.mp4"
    vis_saver.save_skeleton_video(video_path, output_vid, raw_lm)

    # 3. 분석
    analyzer = PoseAnalyzer()
    coords_smooth = np.zeros_like(coords)
    for i in range(coords.shape[1]):
        for j in range(2): coords_smooth[:, i, j] = analyzer.smooth_signal(coords[:, i, j])

    df_all, df_filtered = analyzer.calculate_angles_and_filter(
        coords_smooth, vis, view_mode=view_mode
    )

    # 4. 가려짐 로그 출력
    if view_mode == 'side':
        print(f"\n🔍 [Occlusion Log] 가려짐 대체 내역 ({len(analyzer.replacement_logs)}건):")
        if analyzer.replacement_logs:
            for log in analyzer.replacement_logs[:3]: print("  - " + log)
            if len(analyzer.replacement_logs) > 3: print("  ... (생략)")
        else:
            print("  - 가려짐 대체가 발생하지 않았습니다.")

    # 5. Rep 분리 (항상 knee_mean 기준)
    knee_seq = analyzer.smooth_signal(df_all['knee_mean'].values)
    detector = RepDetector(min_delta=10.0)
    reps = detector.detect(knee_seq)

    print(f"\n✅ [Result] 총 {len(reps)}개의 Rep이 감지되었습니다.")

    results = []
    for i, r in enumerate(reps):
        s, e = r['start'], r['end']
        rep_data = analyzer.resample_rep(df_filtered.iloc[s:e+1])

        results.append({
            "rep_id": i+1,
            "view_mode": view_mode,
            "frames": (s, e),
            "data": rep_data
        })
        print(f"  - Rep {i+1}: Frame {s}~{e}, Min Depth: {r['min_angle']:.1f}°")

    return results

In [5]:
# =========================
# Drive mount (Colab)
# =========================
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# =========================================================
# 6. 실행 가이드
# =========================================================
"""
[실행 방법]
1. 위 코드를 복사하여 실행합니다.
2. 아래 `USER_VIDEO_PATH`에 분석할 영상 파일의 경로를 입력합니다.
3. `VIEW_MODE`를 'side' (측면) 또는 'front' (정면)으로 설정합니다.
"""

# ▼▼▼ [실행 코드] ▼▼▼

USER_VIDEO_PATH = "/content/drive/MyDrive/parrot/videos/정면.mp4"
VIEW_MODE = 'front' # 'side' or 'front'

if os.path.exists(USER_VIDEO_PATH):
    final_data = process_user_video(USER_VIDEO_PATH, view_mode=VIEW_MODE)

    if final_data:
        rep1 = final_data[0]['data']
        print(f"\n📊 [Rep 1] 데이터 확인 (Mode: {VIEW_MODE})")
        print(f"   - Shape: {rep1.shape} (Frames, Features)")
        print(f"   - Columns ({len(rep1.columns)}개): {list(rep1.columns)}")

        print("\n👉 데이터 미리보기:")
        pd.set_option('display.max_columns', None)
        print(rep1.head(3))
else:
    print(f"⚠️ 파일 없음: {USER_VIDEO_PATH}")


🎬 [Start] 영상 분석 시작 (Mode: front): 정면.mp4
🎥 [Saved] 스켈레톤 영상 저장 완료: /content/drive/MyDrive/parrot/videos/정면_front_skeleton.mp4

✅ [Result] 총 4개의 Rep이 감지되었습니다.
  - Rep 1: Frame 147~365, Min Depth: 135.0°
  - Rep 2: Frame 467~681, Min Depth: 139.6°
  - Rep 3: Frame 773~980, Min Depth: 140.6°
  - Rep 4: Frame 1066~1288, Min Depth: 137.4°

📊 [Rep 1] 데이터 확인 (Mode: front)
   - Shape: (64, 7) (Frames, Features)
   - Columns (7개): ['fppa_left', 'fppa_right', 'fppa_mean', 'pelvic_tilt', 'trunk_lateral_tilt', 'knee_width_ratio', 'head_pitch_proxy']

👉 데이터 미리보기:
    fppa_left  fppa_right   fppa_mean  pelvic_tilt  trunk_lateral_tilt  \
0  177.388834  179.642134  178.515484   179.314995          178.419609   
1  177.073543  179.516486  178.295015   179.476116          178.599725   
2  176.719165  178.270256  177.494710   179.294477          178.971997   

   knee_width_ratio  head_pitch_proxy  
0          0.753675          0.729052  
1          0.762782          0.726378  
2          0.77189

In [ ]:
import json

def save_rep_to_json(rep_data, output_path, view_mode='side'):
    """
    분석된 DataFrame 데이터를 최종 AI 코드가 읽을 수 있는 JSON 형태로 저장
    """
    # DataFrame -> List of Dictionaries 변환
    records = rep_data.to_dict(orient='records')

    # 최종 포맷 맞추기
    json_output = {
        "source_video": os.path.basename(output_path),
        "view_mode": view_mode,
        "frames": []
    }

    for row in records:
        # AI 모델은 'angles' 키 안에 있는 데이터를 찾음
        frame_data = {
            "angles": row  # {'knee_left': 170.1, ...} 형태
        }
        json_output["frames"].append(frame_data)

    # 파일 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(json_output, f, indent=2)

    print(f"💾 [Saved] AI 입력용 JSON 저장 완료: {output_path}")

# ---------------------------------------------------------
# [실행부] 위에서 분석한 final_data를 JSON으로 저장하기
# ---------------------------------------------------------
# ★ 주의: 위쪽 셀의 process_user_video가 먼저 실행되어 final_data가 있어야 함

if 'final_data' in locals() and final_data:
    # 감지된 모든 Rep을 각각 JSON으로 저장
    for rep in final_data:
        rep_id = rep['rep_id']
        df_rep = rep['data']
        mode = rep['view_mode']

        # 파일명 자동 생성 (예: my_video_side_rep1.json)
        base_name = os.path.splitext(os.path.basename(USER_VIDEO_PATH))[0]
        json_filename = f"{base_name}_{mode}_rep{rep_id}.json"
        save_path = os.path.join("/content/drive/MyDrive/parrot/videos", json_filename)

        save_rep_to_json(df_rep, save_path, view_mode=mode)
else:
    print("⚠️ 저장할 분석 데이터가 없습니다. 영상을 먼저 분석해주세요.")

⚠️ 저장할 분석 데이터가 없습니다. 영상을 먼저 분석해주세요.
